# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [49]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import os

# Read the CSV and Perform Basic Data Cleaning

In [52]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]



In [76]:
# Load the data
#file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv('C://Users/Willi/Desktop/machine_learning_homework/Resources/LoanStats_2019Q1.csv.zip', skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)


df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 86 columns):
loan_amnt                     68817 non-null float64
int_rate                      68817 non-null float64
installment                   68817 non-null float64
home_ownership                68817 non-null object
annual_inc                    68817 non-null float64
verification_status           68817 non-null object
issue_d                       68817 non-null object
loan_status                   68817 non-null object
pymnt_plan                    68817 non-null object
dti                           68817 non-null float64
delinq_2yrs                   68817 non-null float64
inq_last_6mths                68817 non-null float64
open_acc                      68817 non-null float64
pub_rec                       68817 non-null float64
revol_bal                     68817 non-null float64
total_acc                     68817 non-null float64
initial_list_status           68817 non-null obj

In [85]:
#df.home_ownership.value_counts()
#df.verification_status.value_counts()
#df.issue_d.value_counts()
#df.loan_status.value_counts()
#df.pymnt_plan.value_counts()
#df.initial_list_status.value_counts()
#df.next_pymnt_d.value_counts()
df.application_type.value_counts()
#df.hardship_flag.value_counts()
#df.debt_settlement_flag.value_counts()

Individual    59206
Joint App      9611
Name: application_type, dtype: int64

# Split the Data into Training and Testing

In [45]:
# Create our features
X = df.drop(columns = 'loan_status')

# Create our target
y = df.loan_status

In [86]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 85 columns):
loan_amnt                     68817 non-null float64
int_rate                      68817 non-null float64
installment                   68817 non-null float64
home_ownership                68817 non-null object
annual_inc                    68817 non-null float64
verification_status           68817 non-null object
issue_d                       68817 non-null object
pymnt_plan                    68817 non-null object
dti                           68817 non-null float64
delinq_2yrs                   68817 non-null float64
inq_last_6mths                68817 non-null float64
open_acc                      68817 non-null float64
pub_rec                       68817 non-null float64
revol_bal                     68817 non-null float64
total_acc                     68817 non-null float64
initial_list_status           68817 non-null object
out_prncp                     68817 non-null flo

In [7]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [8]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [9]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE

Counter({'low_risk': 51366, 'high_risk': 51366})

In [10]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=1, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [11]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.7163908158823367

In [12]:
# Display the confusion matrix
# YOUR CODE HERE

array([[   73,    28],
       [ 4960, 12144]])

In [13]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.72      0.71      0.03      0.72      0.51       101
   low_risk       1.00      0.71      0.72      0.83      0.72      0.51     17104

avg / total       0.99      0.71      0.72      0.82      0.72      0.51     17205



### SMOTE Oversampling

In [ ]:
# Resample the training data with SMOTE
# YOUR CODE HERE

Counter({'low_risk': 51366, 'high_risk': 51366})

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=1, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.7001170474858525

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

array([[   71,    30],
       [ 5178, 11926]])

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.70      0.70      0.03      0.70      0.49       101
   low_risk       1.00      0.70      0.70      0.82      0.70      0.49     17104

avg / total       0.99      0.70      0.70      0.82      0.70      0.49     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE

Counter({'high_risk': 246, 'low_risk': 246})

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=1, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.6433412021043078

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

array([[  82,   19],
       [8983, 8121]])

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.81      0.47      0.02      0.62      0.40       101
   low_risk       1.00      0.47      0.81      0.64      0.62      0.37     17104

avg / total       0.99      0.48      0.81      0.64      0.62      0.37     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE

Counter({'high_risk': 51361, 'low_risk': 46653})

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=1, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

0.69732081662329

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

array([[   72,    29],
       [ 5443, 11661]])

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.71      0.68      0.03      0.70      0.49       101
   low_risk       1.00      0.68      0.71      0.81      0.70      0.48     17104

avg / total       0.99      0.68      0.71      0.81      0.70      0.48     17205

